<a href="https://colab.research.google.com/github/konankd/Naive-Bayes-Classifier-Development-and-Evaluation/blob/main/Naive_Bayes_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from csv import reader
from csv import writer
import random

def loadCSV(fileName):
    dataSet = list()
    with open(fileName, 'r') as f:
        csvReader = reader(f)
        for row in csvReader:
            if not row:
                continue
            row = " ".join(row)
            row = row.replace(":", " ")
            dataSet.append(row.split(" "))
    return dataSet

def loadFile(fileName, data):
    data = loadCSV(fileName)
    if data[0] != [row[0] for row in data]:
        random.shuffle(data)
        return data
    else:
        data = data[1:]
        random.shuffle(data)
        return data[1:]

def calculatePosteriorProbabilities(metaData, data):
    posteriorProbabilities = dict()
    classFeatres = dict()
    for idx, ele in enumerate(metaData):
        classFeatres[idx] = ele[0]

    for val in metaData:
        k = val[0]
        posteriorProbabilities[k] = dict.fromkeys(val[1:])
    for i in posteriorProbabilities.keys():
        for j in posteriorProbabilities[i].keys():
            posteriorProbabilities[i][j] = 1
    for row in data:
        for index, element in enumerate(row[:-1]):
            posteriorProbabilities[classFeatres[index]][element]+=1
    for i in posteriorProbabilities.keys():
        for j in posteriorProbabilities[i].keys():
            posteriorProbabilities[i][j] = posteriorProbabilities[i][j]/(len(data)+1)
    return posteriorProbabilities

def calculate_prior_probabilities(dataSet, datasetLength):
    prior_probabilies = dict.fromkeys(dataSet.keys())
    for key in dataSet:
        a = len(dataSet[key])/datasetLength
        prior_probabilies[key] = a
    return prior_probabilies

class NaiveBayes:

    def calPriorPostriorProbabilities(self,trainSet, metaData):
        sep = dict()
        for i in range(len(trainSet)):
            if (trainSet[i][-1] not in sep):
                sep[trainSet[i][-1]] = list()
            sep[trainSet[i][-1]].append(trainSet[i])
        priorProbabilities = calculate_prior_probabilities(sep, len(trainSet))
        postieriorProbabilities = dict()
        for key in sep.keys():
            postieriorProbabilities[key] = calculatePosteriorProbabilities(metaData[:-1], sep[key])

        return priorProbabilities, postieriorProbabilities

    def classify(self,testDataSet, metaData, priorProbabilities, postieriorProbabilities):
        classFeatres = dict()
        for index, element in enumerate(metaData[:-1]):
            classFeatres[index] = element[0]

        class_index = dict()

        final_prob = dict.fromkeys(metaData[-1][1:])
        for index, element in enumerate(metaData[-1][1:]):
            class_index[element] = index

        out_filename = input("\nInput name of file to store the classified output : ")
        f = open(out_filename,'w+')
        Writer = writer(f)
        confusionMatrix = list()
        cnt = 0
        for i in range(len(metaData[-1][1:])):
            row = []
            for j in range(len(metaData[-1][1:])):
                row.append(0)
            confusionMatrix.append(row)

        for row in testDataSet:
            for key in final_prob.keys():
                prob = 1
                for index, ele in enumerate(row[:-1]):
                    prob*=postieriorProbabilities[key][classFeatres[index]][ele]
                final_prob[key] = prob*priorProbabilities[key]

            max_value = max(final_prob, key=final_prob.get)
            if max_value == row[-1]:
                cnt = cnt + 1
            confusionMatrix[metaData[-1][1:].index(row[-1])][metaData[-1][1:].index(max_value)] += 1
            towrite = row[:-1]+[max_value]
            Writer.writerow(towrite)
        f.close()
        print(confusionMatrix)

    def KFold(self, train_data, metaData, k):
        # print(train_data, metaData, k)
        avg = 0;
        for i in range(k):
            trainset = list()
            testset = list()
            block = len(train_data) / k
            block = int(block)
            for j in range(len(train_data)):
                if ((i * block <= j) and (j <= (i+1) * block)):
                    testset.append(train_data[j])
                else :
                    trainset.append(train_data[j])
            pr, po = self.calPriorPostriorProbabilities(trainset, metaData)
            acc = self.naive_acc(testset, metaData, pr, po)
            avg += acc
            print("Fold " , i, " " , acc)
        print("AVG ACC IS ", avg / k)
            # print(acc)


    def naive_acc(self, testDataSet, metaData, priorProbabilities, postieriorProbabilities):
        count = 0
        final_prob = dict.fromkeys(metaData[-1][1:])
        classFeatres = dict()
        for index, element in enumerate(metaData[:-1]):
            classFeatres[index] = element[0]
            final_prob = dict.fromkeys(metaData[-1][1:])
        class_index = dict()
        for index, element in enumerate(metaData[-1][1:]):
                class_index[element] = index


        for row in testDataSet:
            for key in final_prob.keys():
                prob = 1
                for index, ele in enumerate(row[:-1]):
                    prob*=postieriorProbabilities[key][classFeatres[index]][ele]
                final_prob[key] = prob*priorProbabilities[key]

            max_value = max(final_prob, key=final_prob.get)
            if max_value == row[-1]:
                count+=1
        accuracy = count/len(testDataSet)
        return accuracy

def main():
    prior_probabilities = None
    postieriorProbabilities = None
    meta_filename = ''
    train_filename = ''
    while True:
        print("\n 1. Build the naive bayes model")
        print("\n 2. test the classifier with input file")
        print("\n 3. calculate accuracy for given test file")

        option = int(input("\nSelect option : "))
        print()
        if option == 1:
            classifier = NaiveBayes()
            meta_filename = input("enter meta data file name")
            metaData = loadCSV(meta_filename)
            train_filename = input("enter train file name")
            train_data = loadFile(train_filename, metaData)
            prior_probabilities, postieriorProbabilities = classifier.calPriorPostriorProbabilities(train_data, metaData)
            print("model has been trained now try 2 and 3")
        elif option == 2:
            train_filename = input("enter train file name")
            kvalue = input("enter k for k-fold cross validation")
            kvalue = int(kvalue)
            train_data = loadFile(train_filename, metaData)
            metaData = loadCSV(meta_filename)

            classifier.KFold(train_data, metaData, kvalue)
            # test_filename = input("enter test file name")
            # testDataSet = loadFile(test_filename, metaData)
            # classifier.classify(testDataSet, metaData, prior_probabilities, postieriorProbabilities)
        elif option == 3:
            classifier = NaiveBayes()
            test_filename = input("enter test file name")
            testDataSet = loadFile(test_filename, metaData)

            classifier.classify(testDataSet, metaData, prior_probabilities, postieriorProbabilities)

            acc = classifier.naive_acc(testDataSet, metaData, prior_probabilities, postieriorProbabilities)
            print("acc is : ")
            print(acc)
        else:
            break
main()



 1. Build the naive bayes model

 2. test the classifier with input file

 3. calculate accuracy for given test file

Select option : 1

enter meta data file namecar.meta.txt
enter train file namecar.train.txt
model has been trained now try 2 and 3

 1. Build the naive bayes model

 2. test the classifier with input file

 3. calculate accuracy for given test file

Select option : 2

enter train file namecar.train.txt
enter k for k-fold cross validation3
Fold  0   0.828693790149893
Fold  1   0.860813704496788
Fold  2   0.8479657387580299
AVG ACC IS  0.8458244111349037

 1. Build the naive bayes model

 2. test the classifier with input file

 3. calculate accuracy for given test file
